In [1]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from pathlib import Path
from research_tools.gpu import get_gpus_available

load_dotenv()


hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

n_gpus = 1

gpus_available = get_gpus_available()
n_gpus = min(n_gpus, len(gpus_available))
gpus = gpus_available[:n_gpus]

assert n_gpus > 0, "No GPUs available"

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in gpus])

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq
import pickle

data_dir = Path("../data")
cache_path = data_dir / "full.pickle"

USE_CACHE = True

if USE_CACHE:
    assert cache_path.exists(), "Cache file does not exist"
    with open(cache_path, "rb") as f:
        data = pickle.load(f)
else:
    data = {}
    # iterate over all enums
    for name in Datasets:

        dataset_config = DATASETS_DICT[name]

        def get_dataset(train_files: List[str], val_files: List[str], max_length: int):
            train = local_load_dataset(data_dir, train_files)
            val = local_load_dataset(data_dir, val_files)
            train_records = process_corpus(train, tokenizer, max_length)
            val_records = process_mcq(val, tokenizer, max_length)
            mcq_records = process_mcq(val, tokenizer, max_length, expand_choices=False)
            return {
                "corpus": train_records,
                "mcq": mcq_records,
                "val": val_records,
            }

        max_length = 512

        unlearn_files = dataset_config["unlearn_files"]
        val_unlearn_files = dataset_config["val_unlearn_files"]

        print(f"Processing {name}")

        data[name] = get_dataset(unlearn_files, val_unlearn_files, max_length)

        if "retain" not in data:
            retain_files = dataset_config["retain_files"]
            val_retain_files = dataset_config["val_retain_files"]
            data["retain"] = get_dataset(retain_files, val_retain_files, max_length)

    with open(cache_path, "wb") as f:
        pickle.dump(data, f)

In [ ]:
import torch


def group_shuffle(data: List, group_size: int = 1, perm: Optional[torch.Tensor] = None):
    if perm is None:
        n = len(data) // group_size
        perm = torch.randperm(n)

    res = []
    for i in perm:
        res += data[i * group_size : (i + 1) * group_size]
    return res


def create_k_folds(data: List, k: int, group_size: int = 1):
    n = len(data) // group_size
    fold_size = n // k
    folds = [fold_size] * k
    for i in range(n % k):
        folds[i] += 1

    assert sum(folds) == n

    res = []
    start = 0

    for fold_size in folds:
        start_idx = start * group_size
        end_idx = (start + fold_size) * group_size

        res.append(data[start_idx:end_idx])

        start += fold_size

    return res


def get_folds_shuffled(records: Dict[str, List], k: int):

    perm = torch.randperm(len(records["mcq"]))

    store = [
        {"corpus": c, "mcq": m, "val": v}
        for c, m, v in zip(
            create_k_folds(group_shuffle(records["corpus"], 3, perm=perm), k, 3),
            create_k_folds(group_shuffle(records["mcq"], perm=perm), k),
            create_k_folds(
                group_shuffle(
                    records["val"],
                    4,
                    perm=perm,
                ),
                k,
                4,
            ),
        )
    ]

    return store


records = data[Datasets.WMDP]
k = 3

store = get_folds_shuffled(records, k)

In [ ]:
from relearn.unlearn.rmu import train_rmu
import itertools


def super_rmu(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    forget_records_dict: Dict[str, Dict],
    retain_records_dict: Dict[str, Dict],
    eval_records_dict: Dict[str, Dict],
    k_folds: int,
    lr: float = 1e-5,
    joint_train: bool = False,
    **kwargs,
):
    assert k_folds <= 26, "k_folds must be less than 26"

    def fold_name(i: int):
        return chr(ord("A") + i)

    folds = get_folds_shuffled(forget_records_dict, k_folds)

    def get_data(fold_inds: List[int]):
        if joint_train:
            return {
                fold_name(i): {
                    "corpus": folds[i]["corpus"],
                    "mcq": folds[i]["mcq"],
                    "val": folds[i]["val"],
                }
                for i in range(fold_inds)
            }
        else:
            return {
                fold_name(-1): {
                    k: list(
                        itertools.chain(
                            *[f[k] for i, f in enumerate(folds) if i in fold_inds]
                        )
                    )
                    for k in folds.keys()
                }
            }

    if eval_records_dict is None:
        eval_records_dict = {fold_name(i): folds[i]["val"] for i in range(k_folds)}
        eval_records_dict["retain"] = retain_records_dict

    for i in range(k_folds):
        print(f"Training fold {i}")

        forget_fold_inds = [i]
        retain_fold_inds = list(range(i + 1, k_folds))

        forget_dict = get_data(forget_fold_inds)
        retain_dict = get_data(retain_fold_inds)
        retain_dict["retain"] = retain_records_dict

        model = train_rmu(
            model,
            forget_dict,
            retain_dict,
            eval_records_dict,
            forget_alphas={},
            retain_alphas={},
            lr=lr,
            tokenizer=tokenizer,
            kwargs=kwargs,
        )

    return model